# Fires widgets

This widget takes an Area of Interest (adm0, adm1, adm2) and requests the number of fires recorded in the last X number of days. The service is only able to look back as far as the last 7 days, however.

The widget should simply display a dynamic sentence showing the number of detected fires in the AOI over the last X number of days.

**NOTE**: The service is a little spotty and sometimes needs a few attempts to return a value

In [1]:
#Import Global Metadata etc

%run '0.Importable_Globals.ipynb'

In [2]:
from datetime import date
from datetime import timedelta
import datetime
import math


In [3]:
adm0 = 'BRA'
adm1 = 1
adm2 = None


time_frame = 7 #interested in the last 7 days
threshold = 0  #hardcode this

tags = ["forest_change"]
selectable_polynames = [None]

#metadata slug: widget_fire_alert_location

# Functions

In [4]:
def get_geostore(iso, adm1=None, adm2=None):
    """Returns geostore of adm region. Here, it is only required for adm2 since
    the VIIRS service only has endpoints for adm0 and adm1 regions"""
   
    if adm2:
        print('Request for adm2 area')
        url =f'https://api.resourcewatch.org/v1/geostore/admin/{iso}/{adm1}/{adm2}' 
    elif adm1:
        print('Request for adm1 area')
        url =f'https://api.resourcewatch.org/v1/geostore/admin/{iso}/{adm1}' 
    elif iso:
        print('Request for adm0 area')
        url =f'https://api.resourcewatch.org/v1/geostore/admin/{iso}'
    
    r = requests.get(url)
    data = r.json().get('data').get('id')
    
    return data

In [5]:
def fires_queries(start_date, end_date, iso, adm1=None, adm2=None):
    """Returns the ncorrect url and params for the VIIRS query. Adm2 requires a geostore
    as the VIIRS service only has endpoints for adm0 and adm1 regions"""
    
    if adm2:
        url = f'https://production-api.globalforestwatch.org/v1/viirs-active-fires'
        properties = {
              "geostore" : get_geostore(iso, adm1, adm2),
              "period" : start_date + ',' + end_date,
              "thresh" : threshold}
    elif adm1:
        url = f'https://production-api.globalforestwatch.org/v1/viirs-active-fires'   
        properties = {
              "geostore" : get_geostore(iso, adm1, adm2),
              "period" : start_date + ',' + end_date,
              "thresh" : threshold}
    elif iso:
        url = f'https://production-api.globalforestwatch.org/v1/viirs-active-fires/admin/{iso}'
        properties = {
              "period" : start_date + ',' + end_date,
              "thresh" : threshold,
              "group" : 'true'}
    
    return (url, properties)

In [6]:
def time_period(time_frame=7):
    """Returns a start- and end-date to be passed to query in 'yyyy-mm-dd' format
    where the start date is the current date (today). Defaults to the last week (days=7)"""
    
    today = date.today()
    
    start = today.strftime('%Y-%m-%d')
    end = (today - timedelta(days=time_frame-1)).strftime('%Y-%m-%d') 
    
    return (start, end)

In [7]:
%%time

#Get start and end dates
start_date = time_period(time_frame)[1]
end_date = time_period(time_frame)[0]

#Get url and params for requests
url = fires_queries(start_date, end_date, adm0, adm1, adm2)[0]
params = fires_queries(start_date, end_date, adm0, adm1, adm2)[1]

r = requests.get(url, params)

print(r.url)
print(f'Status: {r.status_code}')

data = r.json().get('data')

pprint(data)

Request for adm1 area
Request for adm1 area
https://production-api.globalforestwatch.org/v1/viirs-active-fires?geostore=a804728313af17a23cb314d2db36e359&period=2018-01-26%2C2018-02-01&thresh=0
Status: 200
{'attributes': {'areaHa': 15270650.9082289,
                'downloadUrls': {'csv': "https://wri-01.cartodb.com/api/v2/sql?q=%0A%20%20%20%20%20%20%20%20SELECT%20pt.*%20%0A%20%20%20%20%20%20%20%20FROM%20vnp14imgtdl_nrt_global_7d%20pt%20%0A%20%20%20%20%20%20%20%20where%20acq_date%20%3E%3D%20'2018-01-26'%0A%20%20%20%20%20%20%20%20%20%20%20%20AND%20acq_date%20%3C%3D%20'2018-02-01'%0A%20%20%20%20%20%20%20%20%20%20%20%20AND%20ST_INTERSECTS(ST_SetSRID(ST_GeomFromGeoJSON('%7B%22type%22%3A%22MultiPolygon%22%2C%22coordinates%22%3A%5B%5B%5B%5B-73.3325119018555%2C-7.324879169464%5D%2C%5B-72.6593475341795%2C-7.62497901916504%5D%2C%5B-72.1731796264648%2C-7.72384119033802%5D%2C%5B-72.005142211914%2C-7.77484607696533%5D%2C%5B-70.9374160766601%2C-8.0190896987915%5D%2C%5B-70.3695220947264%2C-8.14140224

In [8]:
data.get('attributes').get('value')

12

In [9]:
adm1, adm2

(1, None)

In [10]:
#Extract data and ensure it is in correct order
#Also calculate the total number of fires.

counts = []
total_fires = 0

if adm1 or adm2:
    total_fires = data.get('attributes').get('value')

else:
    for d in data:
            counts.append({'count': d.get('attributes').get('value'), 'date': d.get('attributes').get('day')[:10]})
            total_fires += d.get('attributes').get('value')

            #Order list by 'date'
            counts = sorted(counts, key=lambda k: k['date'], reverse=True)

            counts

# Graph

Shows a sparkline representing the number of fires detected in each day over the previous 7 day period.

ONLY FOR ADM0

In [11]:
if adm1 or adm2:
    print('no daily data')
else:

    x_data = []
    y_data = []

    for c in counts:
        y_data.append(c.get('count'))
        x_data.append(c.get('date')[5:10])

    plt.plot(x_data, y_data, 'r--')

    ax = plt.gca()

    ax.set_xlabel('date in past 7 days')
    ax.set_ylabel('fire count')

    plt.show()

no daily data


In [12]:
areaId_to_name = None
if adm2:
    tmp = get_admin2_json(iso=adm0, adm1=adm1)
    areaId_to_name ={}
    for row in tmp:
        areaId_to_name[row.get('adm2')] = row.get('name')
if adm1 and not adm2:
    tmp = get_admin1_json(iso=adm0)
    areaId_to_name={}
    for row in tmp:
        areaId_to_name[row.get('adm1')] = row.get('name')

In [13]:
if adm2:
    print(f"In {areaId_to_name[adm2]} ", end="")
elif adm1: 
    print(f"In {areaId_to_name[adm1]} ", end="")
elif adm0:
    print(f"In {iso_to_countries[adm0]} ", end="")

# Say 'no fires' instead of '0 fires'
if counts == 0:
    counts = 'no'
    
print(f"there were {total_fires} active fires detected in the last 7 days.", end="")

In Acre there were 12 active fires detected in the last 7 days.